In [4]:
import gensim.downloader as api
import gensim
from gensim.models import Phrases
from gensim.models import KeyedVectors
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [5]:
info = api.info()  # show info about available models/datasets

In [6]:
model = api.load("word2vec-google-news-300")  # download the model and return as object ready for use
#w2v = dict(zip(model.index2word, model.vectors))

In [7]:
import re
def remove_empty_string(input_string):
    for e, i in enumerate(input_string):
        if i[-1] == ' ' and input_string[e+1][-1] == ' ':
            input_string[e] = i.rstrip()
    joined_string = ''.join(input_string)
    for e, i in enumerate(joined_string):
        if i == ' ' and joined_string[e+1] == ' ':
            del i
    sentences = nltk.sent_tokenize(joined_string)
    return sentences


In [8]:
raw_string = [" ROME — Defying reports that their planned partnership is ", "doomed to fail", ", France’s Naval Group and ", "Italy’s Fincantieri", " have announced a joint venture to build and export naval vessels. ", " The two ", "state-controlled shipyards", " said they were forming a 50-50 joint venture after months of talks to integrate their activities. The move comes as Europe’s fractured shipbuilding industry faces stiffer global competition. ", " The firms said in a statement that the deal would allow them to “jointly prepare winning offers for binational programs and export market,” as well as create joint supply chains, research and testing. ", " Naval Group and Fincantieri first announced talks on cooperation last year after the latter negotiated a controlling share in French shipyard STX. But the deal was reportedly losing momentum due to resistance from French industry and a political row between France and Italy over migrants. ", " The new deal falls short of the 10 percent share swap predicted by French Economy and Finance Minister Bruno Le Maire earlier this year, and far short of the total integration envisaged by Fincantieri CEO Giuseppe Bono. ", " The statement called the joint venture the “first steps” toward the creation of an alliance that would create “a more efficient and competitive European shipbuilding industry.”", " Naval Group CEO Hervé Guillou, speaking at the Euronaval trade expo in Paris on Oct. 24, said the alliance is based on “two countries sharing a veritable naval ambition.”", " The joint venture is necessary because the “context of the global market has changed drastically,” he added, specifically mentioning new market entrants Russia, China, Singapore, Ukraine, India and Turkey.", "Sign up for the Early Bird Brief, the defense industry's most comprehensive news and information, straight to your inbox.", "By giving us your email, you are opting in to the Early Bird Brief.", " When asked about an initial product to be tackled under the alliance, Guillou acknowledged: “The answer is simple: there is nothing yet.”", " However, the firms said they are working toward a deal to build four logistics support ships for the French Navy, which will be based on an Italian design. ", "Competition flares up for the follow-on portion of a deal previously won by the French shipbuilder.", " The firms also plan to jointly bid next year on work for midlife upgrades for Horizon frigates, which were built by France and Italy and are in service with both navies. The work would include providing a common combat management system. ", " The statement was cautious about future acceleration toward integration. “A Government-to-Government Agreement would be needed to ensure the protection of sovereign assets, a fluid collaboration between the French and Italian teams and encourage further coherence of the National assistance programs, which provide a framework and support export sales,” the statement said.", " But the firms were optimistic the deal would be “a great opportunity for both groups and their eco-systems, by enhancing their ability to better serve the Italian and French navies, to capture new export contracts, to increase research funding and, ultimately, improve the competitiveness of both French and Italian naval sectors.”", " ", "Sebastian Sprenger", " in Paris contributed to this report."]
sentences = remove_empty_string(raw_string)

In [9]:
# The 'skipthoughts' module can be found at the root of the GitHub  repository linked above
#import skipthoughts

# You would need to download pre-trained models first
#model = skipthoughts.load_model()

#encoder = skipthoughts.Encoder(model)
#encoded =  encoder.encode(sentences)
a = model['ROME']
a.shape

(300,)

In [17]:
embeddings = []
stpwrds = stopwords.words('english') + list(string.punctuation) + ['—', '“', '”', "'", "’"]

for i in sentences:
    temp = []
    words = nltk.word_tokenize(i)
    for word in words:
        true_len = len(words) - len([w for w in stpwrds if w in words])
        #if word not in stpwrds:
        if word in model.vocab:
            v = model[word]/true_len
        #else:
        #    v = np.zeros(300,)
        temp.append(v)
        a = sum(temp)
        np_temp = np.array(a)
    #embeddings.append(temp)
    embeddings.append(np_temp)
    
sentence_vectors = np.array(embeddings)
print(sentence_vectors.shape)
print(sentence_vectors[0])
print(sentence_vectors[0].shape)
print(len(sentence_vectors))

(15, 300)
[-0.0308745   0.1653925   0.07571148  0.04756497 -0.06667558 -0.05863687
  0.00710794 -0.13558097  0.15902577  0.09241784 -0.06377341 -0.05293472
 -0.05117068 -0.02844238 -0.19437043  0.09911446 -0.01195757 -0.03752467
  0.02769603  0.01944068 -0.02954258 -0.02309981  0.08139437 -0.10659392
  0.1635742   0.00746353 -0.00046439  0.10255498 -0.05470872  0.00395136
  0.03557553 -0.05517313 -0.1864266  -0.02056089  0.07103299 -0.0096064
 -0.0338294  -0.00986647 -0.02073071  0.01999233  0.05608334  0.03390902
  0.09134076  0.13127334 -0.01280942 -0.1576432  -0.08905559  0.11800617
 -0.05926779  0.16735774 -0.03892452  0.21454987  0.05379585 -0.00946311
 -0.07339012  0.04836109 -0.22361822 -0.1639789   0.02216571 -0.11142631
  0.03989045  0.02302484 -0.10982081 -0.15812384 -0.14470239  0.01417077
  0.02373207  0.04259457 -0.03389441 -0.00816545 -0.13387531 -0.00848919
 -0.03040214  0.16665919 -0.10114718 -0.1206263   0.04823504  0.02946274
  0.03282895  0.12907809  0.00057055 -0.24

In [18]:
sim_mat = np.zeros([len(sentences), len(sentences)])

#for e, i in enumerate(sentence_vectors):
#    a = cosine_similarity(sentence_vectors[e])
sim_mat.shape    

(15, 15)

In [19]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]

In [20]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [21]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [22]:
num_of_sentences = 4
for i in range(num_of_sentences):
    print(ranked_sentences[i][1])

The statement called the joint venture the “first steps” toward the creation of an alliance that would create “a more efficient and competitive European shipbuilding industry.” Naval Group CEO Hervé Guillou, speaking at the Euronaval trade expo in Paris on Oct. 24, said the alliance is based on “two countries sharing a veritable naval ambition.” The joint venture is necessary because the “context of the global market has changed drastically,” he added, specifically mentioning new market entrants Russia, China, Singapore, Ukraine, India and Turkey.Sign up for the Early Bird Brief, the defense industry's most comprehensive news and information, straight to your inbox.By giving us your email, you are opting in to the Early Bird Brief.
But the firms were optimistic the deal would be “a great opportunity for both groups and their eco-systems, by enhancing their ability to better serve the Italian and French navies, to capture new export contracts, to increase research funding and, ultimatel

In [16]:
model.most_similar('machine')

[('machines', 0.7677487730979919),
 ('wringer_washing', 0.5941920280456543),
 ('machinery', 0.5260839462280273),
 ('Automated_teller', 0.5225658416748047),
 ('roundish_pinhole_shape', 0.5103495121002197),
 ('Machine', 0.5063987374305725),
 ('extrude_liquefied_cornmeal', 0.5044564008712769),
 ('ma_chine', 0.4988497197628021),
 ('CNC_lathe', 0.49585121870040894),
 ('Computer_Numerically_Controlled', 0.4957743287086487)]